In [4]:

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

from tqdm import tqdm
from model import Net
from data_loader import get_train_loader
from data_loader import get_test_loader

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
torch.manual_seed(10)
batch_size = 128
EPOCHS=30
LAMBDA=0.1
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}



model = Net().to(device)
summary(model, input_size=(3, 32, 32))



train_losses = []
test_losses = []
train_acc = []
test_acc = []

train_loader = get_train_loader()
test_loader = get_test_loader()

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        y_pred = model(data)

        # Calculate loss
        criteria = nn.CrossEntropyLoss()
        loss = criteria(y_pred, target)
        train_losses.append(loss)

        # Backpropagation
        loss.backward()
        optimizer.step()

        pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc=f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100 * correct / processed:0.2f}')
        train_acc.append(100 * correct / processed)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))


from torch.optim.lr_scheduler import StepLR
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9,nesterov=False)
#scheduler = StepLR(optimizer, step_size=8, gamma=0.1)
for epoch in range(EPOCHS):
    print('Epoch:', epoch+1)
    train(model, device, train_loader, optimizer, epoch)
    #scheduler.step()
    test(model, device, test_loader)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 30, 30]             864
       BatchNorm2d-2           [-1, 32, 30, 30]              64
              ReLU-3           [-1, 32, 30, 30]               0
           Dropout-4           [-1, 32, 30, 30]               0
            Conv2d-5           [-1, 64, 28, 28]          18,432
       BatchNorm2d-6           [-1, 64, 28, 28]             128
              ReLU-7           [-1, 64, 28, 28]               0
           Dropout-8           [-1, 64, 28, 28]               0
            Conv2d-9          [-1, 128, 26, 26]          73,728
      BatchNorm2d-10          [-1, 128, 26, 26]             256
             ReLU-11          [-1, 128, 26, 26]               0
          Dropout-12          [-1, 128, 26, 26]               0
           Conv2d-13          [-1, 230, 24, 24]         264,960
      BatchNorm2d-14          [-1, 230,

  0%|          | 0/782 [00:00<?, ?it/s]

Epoch: 1


Loss=1.848878026008606 Batch_id=781 Accuracy=35.26: 100%|██████████| 782/782 [00:35<00:00, 22.11it/s]
  0%|          | 0/782 [00:00<?, ?it/s]


Test set: Average loss: 1.4330, Accuracy: 4714/10000 (47.14%)

Epoch: 2


...


Test set: Average loss: 0.6785, Accuracy: 8013/10000 (80.13%)

Epoch: 29


Loss=0.7011525630950928 Batch_id=781 Accuracy=91.51: 100%|██████████| 782/782 [00:35<00:00, 22.29it/s]
  0%|          | 0/782 [00:00<?, ?it/s]


Test set: Average loss: 0.6728, Accuracy: 8081/10000 (80.81%)

Epoch: 30


Loss=0.6145514845848083 Batch_id=781 Accuracy=92.04: 100%|██████████| 782/782 [00:34<00:00, 22.54it/s]



Test set: Average loss: 0.7502, Accuracy: 7941/10000 (79.41%)

